In [1]:
exec(open("../header.py", "r").read())

In [93]:
from scipy import linalg
from sklearn.metrics import roc_auc_score

In [30]:
data = pd.read_csv(data_path/"data-logistic.csv", header=None)
display(data.head())

y = data.iloc[:, 0]
X = data.iloc[:, 1:]

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [94]:
def predict(w, x):
    return 1/(1+sp.exp(-sp.dot(x, w)))

In [35]:
def lagr(C, w, X, y):
    return 1/X.shape[0]*( sp.sum( sp.log( 1 + sp.exp(-y*sp.dot(X, w) ) ) ) ) + 1/2*C*sp.linalg.norm(w)**2

In [124]:
def jac_step(k, C, w, X, y):
    return w + k/X.shape[0]*sp.sum( ((1 - 1/(1 + sp.exp(-y*sp.dot(X, w))))[:, sp.newaxis]*y[:, sp.newaxis]*X ), axis=0) - k*C*w

In [157]:
k = 0.1
C = 10
limit = 1E-5
w_init = sp.array([0.0,0.0])

In [197]:
def minimize(k, C, w, X, y):
    counter = 0

    while True:
        new_w = jac_step(k, C, w, X, y)
        dist = sp.linalg.norm(w - new_w)
        w = new_w
        counter += 1
        if dist <= limit or counter > 10_000:
            break
    return w

In [198]:
w_C = minimize(k, C, w_init.copy(), X, y)

In [199]:
w_noC = minimize(k, 0, w_init.copy(), X, y)

In [200]:
@answer
def compare_aucroc(fout):
    w_init = sp.array([0,0])
    
    auc_noC = roc_auc_score(y, predict(w_noC, X))
    auc_C = roc_auc_score(y, predict(w_C, X))
    
    fout.write(f"{auc_noC:.3f} {auc_C:.3f}")
compare_aucroc()

*** OUTPUT (output/01_compare_aucroc.dat) ***
0.927 0.936


In [201]:
def test():
    C = 10
    w = sp.array([0.2, 0.3])
    eps = sp.array([0.0000001, 0])
    display(-(lagr(C, w+eps, X, y) - lagr(C, w, X, y))/sp.linalg.norm(eps))
    display(1/X.shape[0]*sp.sum( ((1 - 1/(1 + sp.exp(-y*sp.dot(X, w))))[:, sp.newaxis]*y[:, sp.newaxis]*X ) - C*w, axis=0))
test()

-2.024268377365246

1   -2.024268
2   -3.094727
dtype: float64